In [1]:
!pip install dtaidistance

     |████████████████████████████████| 797 kB 1.6 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dtaidistance: filename=dtaidistance-2.3.6-cp36-cp36m-linux_x86_64.whl size=888970 sha256=a9fc97293a79e2a12b93d8b1ee4f5b2e00c3f8007ed117f3d3956431084ca180
  Stored in directory: /home/ec2-user/.cache/pip/wheels/30/5f/18/3db8e0572e06cde30543380d60b4511e3802d1481c3a3a2642
Successfully built dtaidistance


In [13]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from collections import defaultdict
from tqdm import tqdm 
from dtaidistance import dtw
from heapq import nlargest

tqdm.pandas()

In [3]:
df_pred = pd.read_csv("../Data/df_pred_ext.csv").drop("Unnamed: 0", axis=1)

In [4]:
df_pred.head()

,candidate_id,job_order,time_between,time_spent,isco_functie_niveau,education,function_id,isco_code4
0,84556,0,37,156,2.0,0.0,937.0,207.0
1,84556,1,23,116,1.0,0.0,811.0,347.0
2,84556,2,23,275,2.0,0.0,937.0,207.0
3,84556,3,1155,4,1.0,0.0,1521.0,343.0
4,84556,4,203,11,1.0,0.0,1521.0,343.0


In [5]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [6]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 465468/465468 [00:11<00:00, 40007.64it/s]


In [7]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1)]

In [8]:
career_paths = career_paths.loc[career_paths.apply(lambda x: x[-1] != x[-2])]

In [9]:
results = defaultdict(lambda: defaultdict(int))

try:
    for i, current in tqdm(enumerate(career_paths.iteritems()), total=len(career_paths)):
        candidate, career = current
        career = career[:-1].astype("double")

        for j in career_paths.iteritems():
            candidate2, career2 = j

            if candidate != candidate2:
                results[candidate][candidate2] = dtw.distance(career, 
                                                              career2.astype("double"), 
                                                              use_c=True, 
                                                              use_pruning=True)
                
        if i == 1000:
            break
                        
except KeyboardInterrupt:
    pass

  1%|          | 1000/113428 [17:35<32:58:26,  1.06s/it]


In [10]:
best_matches = {}

for i in tqdm(results):
    if len(results[i]):
        best_matches[i] = sorted(results[i], key=lambda x: results[i][x])

100%|██████████| 1001/1001 [01:02<00:00, 16.14it/s]


In [34]:
def knn_classifier(best_matches, career_paths, k=1, n=1):
    
    predictions = []
    
    for candidate in best_matches.keys():
        correct = career_paths[candidate][-1]
                
        neighbors = [(career_paths[neighbor][-1], i + 1) for i, neighbor in enumerate(best_matches[candidate][:k])]
        
        weights = defaultdict(int)
        
        for job, weight in neighbors:
            weights[job] += 1/np.log(1 + weight)
            
        predictions.append(correct in nlargest(n, weights, key=weights.get))

    
    return np.mean(predictions)

In [35]:
results = defaultdict(list)

n=5

for k in tqdm(range(1, 20)):
    results["k"].append(k)
    results[f"accuracy@{n}"].append(knn_classifier(best_matches, career_paths, k=k, n=n))
    
pd.DataFrame(results).set_index("k")

100%|██████████| 19/19 [00:01<00:00,  9.96it/s]


,accuracy@5
k,
1,0.070929
2,0.120879
3,0.162837
4,0.175824
5,0.197802
6,0.211788
7,0.218781
8,0.225774
9,0.232767
